# Kaggle: Housing Prices

Kaggle housing prices challenge in Julia.  
This is an attempt to complete the kaggle housing prices challenge, described [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data), with the Julia language.

I do not have any experience with Julia, so I'm relying on tutorials that can be found on [JuliaBox](https://juliabox.com/) to guide me in data processing, plotting, and analysis ([here](https://github.com/JuliaComputing/JuliaBoxTutorials/blob/master/introductory-tutorials/broader-topics-and-ecosystem) they are on GitHub).  
This will mirror other work that I have tried for this challenge that uses Python (with which I'm much more familiar).

This notebook is a continuation of the notebooks featured in the figment `python/` directory.  Here, we explore the same Kaggle [challenge](https://www.kaggle.com/c/house-prices-advanced-regression-techniques),
but we are using `Julia` instead of `python`!

Since this isn't a unique challenge, I'll refer you to the python notebooks for the exploratory analysis.  For this notebook, I will mostly be playing around with `Julia` (trying to learn it) and specifically, the `Flux` module.

To get started, let's clone the repository into JuliaBox so that we have access to the data.

_Using the 'terminal' from JuliaBox, it is possible to clone the repository into your account and then work from the notebook within the repository directly!_

```
cd /home/jrun
git clone https://github.com/yourname/figment.git
cp -R /home/jrun/figment /mnt/juliabox
```

Then you can load this notebook directly from the repo.

First, let's load some modules that we will use.

In [1]:
using Flux        # ML library
using DataFrames  # handling data
using PyPlot      # plotting library with which I'm very familiar

The data is stored in this local repository, so let's access it and start looking at it!